In [1]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 26.1MB/s 
     |████████████████████████████████| 235kB 48.5MB/s 
     |████████████████████████████████| 3.3MB 50.3MB/s 
     |████████████████████████████████| 122kB 53.5MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 1.8MB 44.0MB/s 
     |████████████████████████████████| 8.2MB 25.5MB/s 
     |████████████████████████████████| 1.2MB 40.4MB/s 
     |████████████████████████████████| 81kB 13.0MB/s 
     |████████████████████████████████| 2.3MB 54.5MB/s 
     |████████████████████████████████| 112kB 56.0MB/s 
     |████████████████████████████████| 245kB 58.4MB/s 
     |████████████████████████████████| 133kB 58.4MB/s 
     |████████████████████████████████| 102kB 12.6MB/s 
     |████████████████████████████████| 174kB 54.5MB/s 
     |████████████████████████████████| 4.2MB 49.9MB/s 
     |████████████████████████████████| 112kB 57.5MB/s 
     |████████████████████████████████| 81kB 10.7MB

In [2]:
import json
with open(r"train.json", "r") as read_file:
    train = json.load(read_file)

In [3]:
train

[{'context': 'Mistborn is a series of epic fantasy novels written by American author Brandon Sanderson.',
  'qas': [{'answers': [{'answer_start': 71, 'text': 'Brandon Sanderson'}],
    'id': '00001',
    'is_impossible': False,
    'question': 'Who is the author of the Mistborn series?'}]},
 {'context': 'The first series, published between 2006 and 2008, consists of The Final Empire,The Well of Ascension, and The Hero of Ages.',
  'qas': [{'answers': [{'answer_start': 28, 'text': 'between 2006 and 2008'}],
    'id': '00002',
    'is_impossible': False,
    'question': 'When was the series published?'},
   {'answers': [{'answer_start': 63,
      'text': 'The Final Empire, The Well of Ascension, and The Hero of Ages'}],
    'id': '00003',
    'is_impossible': False,
    'question': 'What are the three books in the series?'},
   {'answers': [],
    'id': '00004',
    'is_impossible': True,
    'question': 'Who is the main character in the series?'}]}]

In [4]:
with open(r"test.json", 'r') as read_file:
  test = json.load(read_file)

In [5]:
test

[{'context': 'The series primarily takes place in a region called the Final Empire on a world called Scadrial, where the sun and sky are red, vegetation is brown, and the ground is constantly being covered under black volcanic ashfalls.',
  'qas': [{'answers': [{'answer_start': 38,
      'text': 'region called the Final Empire'},
     {'answer_start': 74, 'text': 'world called Scadrial'}],
    'id': '00001',
    'is_impossible': False,
    'question': 'Where does the series take place?'}]},
 {'context': '"Mistings" have only one of the many Allomantic powers, while "Mistborns" have all the powers.',
  'qas': [{'answers': [{'answer_start': 21, 'text': 'one'}],
    'id': '00002',
    'is_impossible': False,
    'question': 'How many powers does a Misting possess?'},
   {'answers': [],
    'id': '00003',
    'is_impossible': True,
    'question': 'What are Allomantic powers?'}]}]

In [6]:
with open(r"predictions.json", 'r') as read_file:
  predictions = json.load(read_file)

In [7]:
predictions

[{'context': 'Vin is a Mistborn of great power and skill.',
  'qas': [{'id': '0', 'question': "What is Vin's speciality?"}]}]

In [8]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [9]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [10]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5

In [11]:
!pip install wandb

In [12]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 20,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [13]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [14]:
### Remove output folder
!rm -rf outputs

In [15]:
# Train the model
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 4/4 [00:00<00:00, 45343.83it/s]


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


...

Running Epoch 18 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(20,
 {'correct': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'eval_loss': [0.080810546875,
   0.04779052734375,
   -0.0518798828125,
   -0.1484375,
   -0.2449951171875,
   -0.349609375,
   -0.458251953125,
   -0.57177734375,
   -0.67919921875,
   -0.771484375,
   -0.84521484375,
   -0.91259765625,
   -0.96826171875,
   -1.013671875,
   -1.0537109375,
   -1.087890625,
   -1.11328125,
   -1.134765625,
   -1.1474609375,
   -1.154296875],
  'global_step': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20],
  'incorrect': [2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'similar': [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
  'train_loss': [4.9045305252075195,
   4.926892280578613,
   4.67885684967041,
   4.125033855438232,
   3.6549317836761475,
   2.9741625785827637,
   2.3010499477386475,
   1.912590503692627,
   1.5487217903137207,
   1.1459264755249023,
  

In [16]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
result

{'correct': 0, 'eval_loss': -1.154296875, 'incorrect': 1, 'similar': 2}

In [18]:
# Make predictions with the model
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]

In [19]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 11214.72it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['power', 'great power', 'power and skill']}]
